In [1]:
import tensorflow as tf


#### 1. 设定神经网络的参数

In [2]:
#输入向量大小
INPUT_NODE = 784
OUTPUT_NODE = 10

IMAGE_SIZE = 28
#黑白图片只有1个通道
NUM_CHANNELS = 1
NUM_LABELS = 10

#第一个卷积层的filters number
CONV1_DEEP = 32
#第一层的filter大小
CONV1_SIZE = 5

#第二层的
CONV2_DEEP = 64
CONV2_SIZE = 5

#全连接层节点数
FC_SIZE = 512

#### 2. 定义前向传播的过程

In [3]:
def inference(input_tensor, train, regularizer):
    with tf.variable_scope('layer1-conv1'):
        #filter参数四维，[size,size,channels,num_filters]
        conv1_weights = tf.get_variable(
            "weight", [CONV1_SIZE, CONV1_SIZE, NUM_CHANNELS, CONV1_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
        #bias参数一维,[num_filters]
        conv1_biases = tf.get_variable("bias", [CONV1_DEEP], initializer=tf.constant_initializer(0.0))
        #第一层卷积
        conv1 = tf.nn.conv2d(input_tensor, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
        #激活函数
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))

    with tf.name_scope("layer2-pool1"):
        #池化层
        pool1 = tf.nn.max_pool(relu1, ksize = [1,2,2,1],strides=[1,2,2,1],padding="SAME")
    #至此的输出为[m,14,14,32]
    
    with tf.variable_scope("layer3-conv2"):
        conv2_weights = tf.get_variable(
            "weight", [CONV2_SIZE, CONV2_SIZE, CONV1_DEEP, CONV2_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases = tf.get_variable("bias", [CONV2_DEEP], initializer=tf.constant_initializer(0.0))
        conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))

    with tf.name_scope("layer4-pool2"):
        pool2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        #至此，输出矩阵为[m,7,7,64]
        
        #为了全连接层，需要将[7,7,64]的矩阵变为一维
        reshaped = tf.contrib.layers.flatten(pool2)
        
    
    #第五层为全连接层,512个结点
    with tf.variable_scope('layer5-fc1'):
        fc1_weights = tf.get_variable("weight", [reshaped.shape[1], FC_SIZE],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc1_weights))
        fc1_biases = tf.get_variable("bias", [FC_SIZE], initializer=tf.constant_initializer(0.1))

        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
        #dropout只在训练时使用
        if train: fc1 = tf.nn.dropout(fc1, 0.5)

    #第六层最后一个fc层，softmax激活输出
    with tf.variable_scope('layer6-fc2'):
        fc2_weights = tf.get_variable("weight", [FC_SIZE, NUM_LABELS],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc2_weights))
        fc2_biases = tf.get_variable("bias", [NUM_LABELS], initializer=tf.constant_initializer(0.1))
        logit = tf.matmul(fc1, fc2_weights) + fc2_biases

    return logit